# Investment proposal for Spark Funds

`importing the needed libraries`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

`Reading and investigation of rounds2 csv file`

In [ ]:
rounds2 = pd.read_csv("./Downloads/rounds2.csv", encoding= 'unicode_escape')

In [ ]:
rounds2.head()

In [ ]:
rounds2.shape

In [ ]:
rounds2.info()

`Unique entries based on all the columns of rounds2 dataframe`

In [ ]:
rounds2.nunique()

`Converting the company_permalink column in rounds2 table to lower case
This is needed to match the letter cases between rounds2 table and companies table to avoid unwanted conflicts while merging`

In [ ]:
rounds2['company_permalink'] = rounds2['company_permalink'].str.lower()
rounds2.head()

`Reading and investigation of companies file`

In [ ]:
companies = pd.read_csv("./Downloads/companies.txt", sep="\t", encoding = "ISO-8859-1")

In [ ]:
companies.head()

In [ ]:
companies.shape

In [ ]:
companies.info()

`Finding the unique entries in companies dataframe for all the columns`

In [ ]:
companies.nunique()

`Renaming and lowercasing the permalink column in companies df to match it with rounds2 df`

In [ ]:
companies = companies.rename(columns={"permalink": "company_permalink"})
companies['company_permalink'] = companies['company_permalink'].str.lower()
companies.head()

#### Merging both the data frames
##### Left Merge is chosen here as per the requirement to add all the columns in companies df to rounds2 df

In [ ]:
master_frame = pd.merge(rounds2, companies, how='left', on='company_permalink')

In [ ]:
master_frame.info()

**Treating missing Values post merging both the df rounds2 and company**

In [ ]:
master_frame.isnull().sum()

In [ ]:
# Finding the percentage of missing values for each column
round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2)

- Decision taken to drop "funding_round_code" and "founded_at" for below reasons <br>
- "funding_round_code" is dropped because of its high percentage of Nan Values and i expect that it will be not used in the analysis of this particular problem <br>
- "founded_at" is dropped as 17% missing values are there and also this is expected not to play a role in the current investigation

In [ ]:
master_frame = master_frame.drop('funding_round_code', axis=1)
master_frame = master_frame.drop('founded_at', axis=1)
round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2)

- Still few parameters have Nan less than 10% and "raised_amount_usd" is about 17% <br>
- Removing the rows having more than 3 NULL values

In [ ]:
#master_frame.info()
len(master_frame[master_frame.isnull().sum(axis=1) > 3].index)

In [ ]:
master_frame = master_frame[master_frame.isnull().sum(axis=1) <= 3]
round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2)

- "raised_amount_usd" is the variable which is expected to play a significant role in the analysis
- Hence It is not wise to impute any values to this based on assumptions
- So deciding to remove those entries which have NULL for "raised_amount_usd"
- Also removing the rows having NULL for "category_list"  parameter

In [ ]:
master_frame = master_frame.dropna(subset=['raised_amount_usd'])
master_frame = master_frame.dropna(subset=['category_list'])
print(round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2))

__Outlier treatment__ <br>
`Some analysis on the "raised_amount_usd" parameter to make sure that uncommon values are discovered and treated`
- 95th to 100th percentile values are checked and found that 100th percentile value is 191 times the 99th percentile value
- Hence deciding to drop the 100th percentile

In [ ]:
print(master_frame['raised_amount_usd'].describe())
print(master_frame['raised_amount_usd'].quantile(0.95))
print(master_frame['raised_amount_usd'].quantile(0.96))
print(master_frame['raised_amount_usd'].quantile(0.97))
print(master_frame['raised_amount_usd'].quantile(0.98))
print(master_frame['raised_amount_usd'].quantile(0.99))
print(master_frame['raised_amount_usd'].quantile(1))

`Investigating the "raised_amount_usd" parameter after removing the 100th percentile value`

In [ ]:
master_frame = master_frame[master_frame.raised_amount_usd <= np.percentile(master_frame.raised_amount_usd,99)]
print(round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2))
print(master_frame['raised_amount_usd'].describe())
print(master_frame['raised_amount_usd'].quantile(0.95))
print(master_frame['raised_amount_usd'].quantile(0.96))
print(master_frame['raised_amount_usd'].quantile(0.97))
print(master_frame['raised_amount_usd'].quantile(0.98))
print(master_frame['raised_amount_usd'].quantile(0.99))
print(master_frame['raised_amount_usd'].quantile(1))
print(master_frame.info())

__With this, checkpoint 1 is concluded, summary of activities done are listed below__
1. rounds2 file read into df
2. companies file read into df
3. Companies df merged to rounds2 df
4. Treatment of missing values and outliers done
#### CHECKPOINT 1 Done

In [ ]:
master_frame.head()

### CHECKPOINT 2 Begins

`Plotting the distribution of "raised_amount_usd" to visualise the distribution for the desired fund type`

In [ ]:
master_subset = master_frame.loc[master_frame["funding_round_type"].isin(["seed", "angel", "venture", "private_equity"])]
plt.figure(num=None, figsize=(12, 8), dpi=80, facecolor='w', edgecolor='k')
sns.boxplot(x='funding_round_type', y='raised_amount_usd', data=master_subset)
plt.yscale('log')
plt.show()

__Post treating the outliers and missing values, Mean and Median are calculated for all the 4 desired funding types__

In [ ]:
print(master_frame.loc[master_frame['funding_round_type'] == "seed"]['raised_amount_usd'].describe())
seed_mean = master_frame.loc[master_frame['funding_round_type'] == "seed"]['raised_amount_usd'].mean()
seed_median = master_frame.loc[master_frame['funding_round_type'] == "seed"]['raised_amount_usd'].median()
print("Seed Mean:",seed_mean)
print("Seed Median:",seed_median)

In [ ]:
print(master_frame.loc[master_frame['funding_round_type'] == "angel"]['raised_amount_usd'].describe())
angel_mean=master_frame.loc[master_frame['funding_round_type'] == "angel"]['raised_amount_usd'].mean()
angel_median=master_frame.loc[master_frame['funding_round_type'] == "angel"]['raised_amount_usd'].median()
print("Angel Mean:",angel_mean)
print("Angel Median:",angel_median)

In [ ]:
print(master_frame.loc[master_frame['funding_round_type'] == "venture"]['raised_amount_usd'].describe())
venture_mean=master_frame.loc[master_frame['funding_round_type'] == "venture"]['raised_amount_usd'].mean()
venture_median=master_frame.loc[master_frame['funding_round_type'] == "venture"]['raised_amount_usd'].median()
print("Venture Mean:",venture_mean)
print("Venture Median:",venture_median)

In [ ]:
print(master_frame.loc[master_frame['funding_round_type'] == "private_equity"]['raised_amount_usd'].describe())
private_equity_mean=master_frame.loc[master_frame['funding_round_type'] == "private_equity"]['raised_amount_usd'].mean()
private_equity_median=master_frame.loc[master_frame['funding_round_type'] == "private_equity"]['raised_amount_usd'].median()
print("Private Equity Mean:",private_equity_mean)
print("Private Equity Median:",private_equity_median)

__Best suited investment type for Spark Funds would be Venture funding since the Mean and the Median of Venture Funding lies in the range of 5 and 15 Million USD__

__With this CheckPoint 2 is done, summary is below__
1. Mean and Median has been calculated for all the 4 desired fund types
2. _It was found that Venture Funding is the suitable one for Spark Funds as the mean of the Venture funding alone lies in the required range_
#### CheckPoint 2 Done

#### CheckPoint 3 Begins

`For the chosen funding type, "Venture Funding", segregate the countries that have Venture funding type`

In [ ]:
venture_funded_countries = master_frame.loc[(master_frame.funding_round_type == 'venture')].groupby(['country_code'])
venture_funded_countries

In [ ]:
venture_funded_countries['company_permalink'].describe()

`Sorting the Venture funded countries based on Investment amount`

In [ ]:
pd.set_option('display.max_rows', None)
top9_venture_funded_countries=venture_funded_countries['raised_amount_usd'].sum().sort_values(ascending=False).reset_index()

In [ ]:
top9_venture_funded_countries

`Retaining the top 9 and removing the rest`

In [ ]:
top9_venture_funded_countries=top9_venture_funded_countries.iloc[0:9]

In [ ]:
top9_venture_funded_countries

In [ ]:
type(top9_venture_funded_countries)

`Dropping the investment amount column to have a df containing only the top 9 venture funded countries in descending order`

In [ ]:
top9_venture_funded_countries=top9_venture_funded_countries.drop('raised_amount_usd', axis=1)

In [ ]:
top9_venture_funded_countries

_Next task is to find from the wiki source, the english speaking nations_ <br>
___Assumption made for this task:___ <br>
1. A country is considered English Speaking ___only if___ English is ___de facto or de facto and de jure official language___ in that country
2. ___All___ the countries listed in ___de facto table___ are considered English speaking countries because English is primary in all those nations
3. In Table 2 of the given wiki page, in ___de facto and de jure official language table___, A country is considered english speaking ___only if English is a Primary Language in that country___

#### Scrape the Wiki page for finding the Englidh speaking countries

In [ ]:
import requests, bs4
url = "https://en.wikipedia.org/wiki/List_of_territorial_entities_where_English_is_an_official_language"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

response = requests.get(url, headers=headers)
soup = bs4.BeautifulSoup(response.text, "html.parser")

Below segment extracts the country code from the first two tables in the given wiki with the assumptions listed above <br>
`Expected Output:` <br>
___List of English speaking Nations from Wiki source___

In [ ]:
tables = soup.find_all('table', class_='sortable')
countries_list = []
flag = False
table_count = 0
for table in tables:
    table_count = table_count + 1
    for td in table.findChildren('td'):
        token = td.text
        token = token.strip()
        # Country code is all upper case and the length is 3 characters
        if(token.isupper() and len(token) == 3):
            flag = True
            country_code = token
        if flag == True:
            flag == False
            if token[0] in ['Y']:
                countries_list.append(country_code)
    # Stopping after the first 2 tables
    # As only the first two tables provides the countries with English as Official and Primary
    if table_count == 2:
        break
print("Below are the country codes of English Speaking countries")
print((countries_list))
    

`Top 3 English speaking countries identified from top 9 with the help of English speaking countries discovered above`

In [ ]:
top3_english_speaking_venture_funded_countries = []
for i in range(len(top9_venture_funded_countries)):
    if top9_venture_funded_countries.iloc[i][0] in countries_list:
        top3_english_speaking_venture_funded_countries.append(top9_venture_funded_countries.country_code.iloc[i])
top3_english_speaking_venture_funded_countries=top3_english_speaking_venture_funded_countries[0:3]
top3_english_speaking_venture_funded_countries

### Top 3 Engligh speaking countries in the descending order of total investments made are
#### 1. USA United States of America
#### 2. GBR United Kingdom
#### 3. CAN Canada

#### CheckPoint 3 ends
#### CheckPoint 4 Begins
##### Sector Analysis

`Reading sector_mapping data into a dataframe and a basic look at the data set`

In [ ]:
sector_mapping = pd.read_csv("./Downloads/mapping.csv", encoding= 'unicode_escape')
sector_mapping.info()

In [ ]:
sector_mapping.head()

- The category list content is found to be not correct <br>
- 'na' has been replaced with '0' <br>
- This is corrected to replace '0' with 'na' and converted to lower case so that no mismatch occurs while merging <br>
- Main sectors are extraced from sector_mapping df and parameters are renamed to reflect the actual property

In [ ]:
# Replacing '0' to 'na'
sector_mapping["category_list"] = sector_mapping["category_list"].str.replace('0', 'na').str.lower()
sector_mapping = pd.melt(sector_mapping, id_vars =['category_list'], value_vars =['Manufacturing','Automotive & Sports',
                                                              'Cleantech / Semiconductors','Entertainment',
                                                             'Health','News, Search and Messaging','Others',
                                                             'Social, Finance, Analytics, Advertising']) 
sector_mapping = sector_mapping[~(sector_mapping.value == 0)]
sector_mapping = sector_mapping.drop('value', axis = 1)
sector_mapping = sector_mapping.rename(columns = {"variable":"main_sectors", "category_list":"primary_sector"})

In [ ]:
print(sector_mapping.nunique())

- ___'category_list' will be considered from master frame to map with sectors df <br>___
- ___Incase more categories are found seperated by |, first category is considered as the primary category for that company___

In [ ]:
master_frame["primary_sector"] = master_frame["category_list"].str.split("|").str[0].str.lower()
master_frame.nunique()

`Inner Merging the master_frame and sector_mapping on primary_sector`

In [ ]:
final_merged_frame = pd.merge(master_frame, sector_mapping, how='inner', on='primary_sector')

In [ ]:
final_merged_frame.nunique()

- ___Above is the merged final frame where for each entry, primary sectors are mapped to main sectors <br>___
- ___Primary sector is presented as a seperate column___

##### Checkpoint 4 ends

##### CheckPoint 5 starts
Identified top 3 countries are ___USA, GBR, CAN___ <br>
Seperate data frames are created for each of those countries with the following conditions
1. ___Funding type is Venture___
2. ___Funding range is 5 million to 15 million USD___
3. ___All the columns of master_frame are present in each of these data frames___

Note: Total count of investments and total amount invested for each main sectors are not part of master frame. This shall be calculated foe each country and for each main sector in the respective country's df

`Analysis for First top country - USA`

In [ ]:
usa=final_merged_frame.loc[(final_merged_frame.country_code == 'USA') & (final_merged_frame.funding_round_type == 'venture') & (final_merged_frame.raised_amount_usd >= 5000000) & (final_merged_frame.raised_amount_usd <= 15000000)]
usa.head()

___Total Count of investments and total amount of investment for each main sector calculated and appended as columns___
1. To do the above, "usa" data frame is grouped by main sectors and count on the group performed and appended as columns, this new data frame is then merged with usa data frame
2. Same has been carried out for total amount of investment as well

##### Finally 2 new columns have been added, total count of investments and total amount of investment grouped by main sectors
Companies have the same main sector mapping will have the same values for above two new parameters

In [ ]:
main_sectors_count=usa.groupby(['main_sectors'])['company_permalink'].count().to_frame().reset_index()
main_sectors_count.rename(columns={'company_permalink' : 'count_of_investments'}, inplace=True)
usa=pd.merge(usa, main_sectors_count, how='inner',on='main_sectors')
main_sectors_total_amount_invested=usa.groupby(['main_sectors'])['raised_amount_usd'].sum().to_frame().reset_index()
main_sectors_total_amount_invested.rename(columns={'raised_amount_usd' : 'total_investment_amount'}, inplace=True)
usa=pd.merge(usa, main_sectors_total_amount_invested, how='inner',on='main_sectors')
usa.head()

- ___Summing up the total investments made for each main sector___ <br>
- ___Output of the below will list the top 3 sectors based on investment amount for USA___ <br>

In [ ]:
usa.groupby(['main_sectors'])['raised_amount_usd'].sum().sort_values(ascending=False)

In [ ]:
# Total amount invested across sectors in USA
print("Total amount invested in USA across main sectors")
usa['raised_amount_usd'].sum()

In [ ]:
#Grouping based on Count of investments
usa_sectors_in_count_order=usa.groupby(['main_sectors'])['count_of_investments'].count().sort_values(ascending=False).to_frame().reset_index()
usa_sectors_in_count_order

In [ ]:
print("Total Count of investments in USA across main sectors")
usa.groupby(['main_sectors'])['count_of_investments'].count().sum()

In [ ]:
print("Company received highest investment in the top sector count wise")
usa.loc[usa['main_sectors'] == usa_sectors_in_count_order['main_sectors'].values[0]].sort_values(by='raised_amount_usd', ascending=False)['name'].values[0]

In [ ]:
print("Company received highest investment in the second top sector count wise")
usa.loc[usa['main_sectors'] == usa_sectors_in_count_order['main_sectors'].values[1]].sort_values(by='raised_amount_usd', ascending=False)['name'].values[0]

__Same procedure mentioned for USA is being carried out for GBR__

In [ ]:
gbr=final_merged_frame.loc[(final_merged_frame.country_code == 'GBR') & (final_merged_frame.funding_round_type == 'venture') & (final_merged_frame.raised_amount_usd >= 5000000) & (final_merged_frame.raised_amount_usd <= 15000000)]
gbr.head()

In [ ]:
main_sectors_count=gbr.groupby(['main_sectors'])['company_permalink'].count().to_frame().reset_index()
main_sectors_count.rename(columns={'company_permalink' : 'count_of_investments'}, inplace=True)
gbr=pd.merge(gbr, main_sectors_count, how='inner',on='main_sectors')
main_sectors_total_amount_invested=gbr.groupby(['main_sectors'])['raised_amount_usd'].sum().to_frame().reset_index()
main_sectors_total_amount_invested.rename(columns={'raised_amount_usd' : 'total_investment_amount'}, inplace=True)
gbr=pd.merge(gbr, main_sectors_total_amount_invested, how='inner',on='main_sectors')
gbr.head()

- ___Summing up the total investments made for each main sector___ <br>
- ___Output of the below will list the top 3 sectors based on investment amount for GBR___ <br>

In [ ]:
gbr.groupby(['main_sectors'])['raised_amount_usd'].sum().sort_values(ascending=False)

In [ ]:
print("Total amount invested in GBR across main sectors")
gbr['raised_amount_usd'].sum()

In [ ]:
#Grouping based on Count of investments
gbr_sectors_in_count_order=gbr.groupby(['main_sectors'])['count_of_investments'].count().sort_values(ascending=False).to_frame().reset_index()
gbr_sectors_in_count_order

In [ ]:
print("Total Count of investments in GBR across main sectors")
gbr.groupby(['main_sectors'])['count_of_investments'].count().sum()

In [ ]:
print("Company received highest investment in the top sector count wise")
gbr.loc[gbr['main_sectors'] == gbr_sectors_in_count_order['main_sectors'].values[0]].sort_values(by='raised_amount_usd', ascending=False)['name'].values[0]

In [ ]:
print("Company received highest investment in the second top sector count wise")
gbr.loc[gbr['main_sectors'] == gbr_sectors_in_count_order['main_sectors'].values[1]].sort_values(by='raised_amount_usd', ascending=False)['name'].values[0]

___Same Procedure as the above two countries are being carried out for Canada as well___

In [ ]:
can=final_merged_frame.loc[(final_merged_frame.country_code == 'CAN') & (final_merged_frame.funding_round_type == 'venture') & (final_merged_frame.raised_amount_usd >= 5000000) & (final_merged_frame.raised_amount_usd <= 15000000)]
can.head()

In [ ]:
main_sectors_count=can.groupby(['main_sectors'])['company_permalink'].count().to_frame().reset_index()
main_sectors_count.rename(columns={'company_permalink' : 'count_of_investments'}, inplace=True)
can=pd.merge(can, main_sectors_count, how='inner',on='main_sectors')
main_sectors_total_amount_invested=can.groupby(['main_sectors'])['raised_amount_usd'].sum().to_frame().reset_index()
main_sectors_total_amount_invested.rename(columns={'raised_amount_usd' : 'total_investment_amount'}, inplace=True)
can=pd.merge(can, main_sectors_total_amount_invested, how='inner',on='main_sectors')
can.head()

- ___Summing up the total investments made for each main sector___ <br>
- ___Output of the below will list the top 3 sectors based on investment amount for Canada___ <br>

In [ ]:
can.groupby(['main_sectors'])['raised_amount_usd'].sum().sort_values(ascending=False)

In [ ]:
print("Total amount invested in Canada across main sectors")
can['raised_amount_usd'].sum()

In [ ]:
#Grouping based on Count of investments
can_sectors_in_count_order=can.groupby(['main_sectors'])['count_of_investments'].count().sort_values(ascending=False).to_frame().reset_index()
can_sectors_in_count_order

In [ ]:
print("Total Count of investments in Canada across main sectors")
can.groupby(['main_sectors'])['count_of_investments'].count().sum()

In [ ]:
print("Company received highest investment in the top sector count wise")
can.loc[can['main_sectors'] == can_sectors_in_count_order.main_sectors.values[0]].sort_values(by='raised_amount_usd', ascending=False)['name'].values[0]

In [ ]:
print("Company received highest investment in the second top sector count wise")
can.loc[can['main_sectors'] == can_sectors_in_count_order.main_sectors.values[1]].sort_values(by='raised_amount_usd', ascending=False)['name'].values[0]

#### Summary of Results
Questions|USA|GBR|Canada
-|-|-|-
1. Total number of investments (count)|12063|621|422
2. Total amount of investment (USD)|107757097294|5379078691|3599289960
3. Top sector (based on count of investments)|Others|Others|Cleantech / Semiconductors
4. Second-best sector (based on count of investments)|Social, Finance, Analytics, Advertising|Social, Finance, Analytics, Advertising|Others
5. Third-best sector (based on count of investments)|Cleantech / Semiconductors|Cleantech / Semiconductors|Social, Finance, Analytics, Advertising
6. Number of investments in the top sector|2950|147|112
7. Number of investments in the second-best sector|2714|133|109
8. Number of investments in the third-best sector|2350|130|78
9. For the top sector count-wise (point 3), which company received the highest investment?|1stdibs|Tribold|Zymeworks
10. For the second-best sector count-wise (point 4), which company received the highest investment?|SigFig|OpenGamma|QuickPlay Media

### CheckPoint 5 ends
### Checkpoint 6 starts

### Plot 1
___Requirement is to find the fraction of total investments for the four intrested funding types, "seed", "angel", "venture" and "private_equity" along with their respective mean___<br><br>
___Steps followed for plotting Plot 1___
1. Create a df containing the fraction of total investments for all the funding types
2. Add a column in the above dataframe containing the mean value for investments in the respective funding types
3. Restrict the above data frame to contain only the four interested funding typoe, seed, angel, ventire and private equity
4. Plot a bar graph where in the two sub plots, fraction of total investment and mean investment amounts are plotted respectively 

In [ ]:
final_merged_frame.head()

In [ ]:
final_merged_frame.groupby(final_merged_frame['funding_round_type'])['raised_amount_usd'].sum()

In [ ]:
final_merged_frame['raised_amount_usd'].sum()

In [ ]:
plot1=(final_merged_frame.groupby(final_merged_frame['funding_round_type'])['raised_amount_usd'].sum()/final_merged_frame['raised_amount_usd'].sum()).to_frame()
plot1['average']=final_merged_frame.groupby(final_merged_frame['funding_round_type'])['raised_amount_usd'].mean()
plot1=plot1.reset_index()
plot1=plot1.rename(columns={'raised_amount_usd':'fraction_of_total_investment', 'average' : 'average_investment'})
plot1=plot1.loc[plot1.funding_round_type.isin(['seed','angel','venture','private_equity'])]
plot1

In [ ]:
plt.figure(figsize=(14, 8))
plt.subplot(1, 2, 1)
sns.barplot(x='funding_round_type', y='fraction_of_total_investment', data=plot1).set_title("Fraction of the Investments")
plt.xlabel("Funding Types")
plt.ylabel("Fraction of Total Investment Amount")

ax=plt.subplot(1, 2, 2)
sns.barplot(x='funding_round_type', y='average_investment', data=plot1).set_title("Mean of the Investments")
plt.xlabel("Funding Types")
plt.ylabel("Mean Investment Amount (in USD) (In 10s of millions)")
ax.axhline(5000000)
#ax1.set_label="5 Million USD"
ax.text(1.02, 5000000, "5 Million USD", va='center', ha="left", bbox=dict(facecolor="w",alpha=0.5),
        transform=ax.get_yaxis_transform())
ax.axhline(15000000)
ax.text(1.02, 15000000, "15 Million USD", va='center', ha="left", bbox=dict(facecolor="w",alpha=0.5),
        transform=ax.get_yaxis_transform())
plt.show()

Following inference made from above Plots
1. Venture funding received the highest fraction of investments, or Venture funding is preferred by most of the investments made
2. Venture funding mean investment amount lies within the range of Spark Investment amount criteria, ie between 5 to 15 million USD <br><br>
___From the above points, Venture funding is the best suited for Spark Funds___

### Plot 2
___Requirement for plot 2 is have a plot showing the top 9 countries against the total amount of investments of funding type "venture"___ <br>
___Steps followed___ <br>
1. Create a df with all the country codes and the amount invested in each of those countries
2. Restrict the above df to have only the top 9 countries in venture funding those have been identified already
3. Plot a bar graph for the countries and the amount invested in those countries

In [ ]:
top9_venture_funded_countries

In [ ]:
#Grouping the master data frame by country code, ordering by investment amount, converting to frame
plot2=final_merged_frame.loc[final_merged_frame['funding_round_type'] == "venture"].groupby(final_merged_frame['country_code'])['raised_amount_usd'].sum().to_frame().reset_index()
#Restricting it to have only top 9 countries
plot2= plot2.loc[plot2.country_code.isin(top9_venture_funded_countries.country_code)]
plot2.sort_values(by='raised_amount_usd', ascending=False)

In [ ]:
plt.figure(figsize=(14, 8))
sns.barplot(x='country_code', y='raised_amount_usd', data=plot2).set_title("Amount Invested in top 9 countries")
plt.xlabel("Country Code")
plt.ylabel("Total amount invested (In USD)")
plt.show()

___From Plot 2, following inference could be made___
1. USA is clearly the most heavily invested country in venture funding
2. Since China and India are not considered as english speaking nations, those are not considered in the analysis
3. GBR and Canada are the 2nd and 3rd countries respectively considering those are English Speaking countries

### Plot 3
___Requirement: A plot showing the number of investments in the top 3 sectors of the top 3 countries on one chart for Venture funding___ <br>
___Steps followed:___
1. DFs of the 3 countries created above are concatenated (in order to have the total count and total amount of investment parameters)
2. DF is already restricted to have only venture type data since the respective country Dataframes contain only venture data
3. DF will be restricted to have data only for the top 3 sectors of all the 3 countries

In [ ]:
sectors_top3=['Cleantech / Semiconductors','Social, Finance, Analytics, Advertising','Others']
plot3=pd.concat([usa,gbr,can])
plot3=plot3.loc[(plot3.main_sectors.isin(sectors_top3))]
plot3.head()

In [ ]:
plt.figure(figsize=(14, 8))
sns.barplot(x='count_of_investments', y='country_code', hue='main_sectors', data=plot3).set_title("Investments in top 3 countries in top 3 sectors")
plt.xlabel("Count of Investments")
plt.ylabel("Country Code")
plt.show()

___From Plot 3, following inference could be made___
1. Top 3 sectors for all top 3 countries are captured
3. In the top 3 cuntries identified, the top 3 sectors are "Cleantech / Semicondictors", "Social, Finance, Analytics, Advertising" and "Others"

### Inference
##### 1. The best suited fund type for Spark Funds is "Venture Funding"
##### 2. Top 3 English speaking nations where majority of the investments happening in Venture Type are USA, Great Britain and Canada
##### 3. Top 3 sectors in the identified top 3 countries are "Cleantech / Semiconductors", "Social, Finance, Analytics, Advertising" and "Others". These are the sectors where maximum investments have happened 

## Proposal (Based on where most investments are happening)
#### Proposed type of Funding: ___Venture Funding___
#### Proposed Countries for investment based on the trend:
- ___USA___
- ___Great Britain___
- ___Canada___

#### Proposed Sectors for investment based on the trend:
- ___Others___
- ___Social, Finance, Analytics, Advertising___
- ___Cleantech / Semiconductors___